# IREI: Search Engines & Real World Data
### Víctor Morcuende Castell and Guillermo Nájera Lavid
#### Course 2022-2023

In [1]:
import pandas as pd

df = pd.read_excel('loinc_dataset-v2.xlsx')
df = pd.concat([df], ignore_index=True)
print(df)

    GLUCOSE IN BLOOD loinc_num  \
0                NaN    1988-5   
1                NaN    1959-6   
2                NaN   10331-7   
3                NaN   18998-5   
4                NaN    1975-2   
..               ...       ...   
62               NaN   54439-5   
63               NaN   18878-9   
64               NaN   18928-2   
65               NaN   29265-6   
66               NaN   23658-8   

                                     long_common_name  \
0   C reactive protein [Mass/volume] in Serum or P...   
1                 Bicarbonate [Moles/volume] in Blood   
2                                  Rh [Type] in Blood   
3      Trimethoprim+Sulfamethoxazole [Susceptibility]   
4    Bilirubin.total [Mass/volume] in Serum or Plasma   
..                                                ...   
62                Calcium bilirubinate/Total in Stone   
63                         Cefazolin [Susceptibility]   
64                        Gentamicin [Susceptibility]   
65  Calcium [Moles/vo

In [2]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler

# Convert queries and markers into numerical features
encoder = OneHotEncoder(sparse=False)
queries = encoder.fit_transform(df['GLUCOSE IN BLOOD'].values.reshape(-1, 1))
markers = encoder.fit_transform(df['long_common_name'].values.reshape(-1, 1))

# Normalize feature values
scaler = StandardScaler()
features = scaler.fit_transform(df.drop(['GLUCOSE IN BLOOD', 'long_common_name', 'relevant'], axis=1))

/Users/victor/miniconda/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/Users/victor/miniconda/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


ValueError: could not convert string to float: '1988-5'

In [ ]:
from sklearn.model_selection import train_test_split

X = pd.concat([pd.DataFrame(queries), pd.DataFrame(markers), pd.DataFrame(features)], axis=1)
y = df['relevant'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from ranklib.models import AdaRank
from ranklib.features import NormalizedDiscountedCumulativeGain as NDCG

# Train AdaRank model
model = AdaRank()
model.fit(X_train, y_train)

# Evaluate model on test set
ndcg = NDCG(k=10)
ndcg_score = ndcg.score(model, X_test, y_test)
print(f'NDCG@10 score: {ndcg_score:.4f}')